In [1]:
import pandas as pd

# Step 2: Read the CSV file into a DataFrame
df = pd.read_excel('/Users/marclambertes/Downloads/ERE2324 - Pass.xlsx')

# List of typeID values you want to count
typeIDs_to_count = [1, 3, 4, 7, 8, 9, 13, 14, 15, 16, 17, 44, 45, 49, 50, 51, 60, 61 ]  # Add the typeID values you want to count

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['contestantId'])
#results_df = pd.DataFrame(columns=['playerName'])

# Group the original DataFrame by 'contestantId' and count occurrences of each 'typeId'
grouped = df.groupby('contestantId')['typeId'].value_counts().unstack(fill_value=0).reset_index()
#grouped = df.groupby('playerName')['typeId'].value_counts().unstack(fill_value=0).reset_index()

# Create a mapping dictionary for typeID to names
typeID_to_name = {
    1: "Pass",
    3: "Take On",
    4: "Foul",
    7: "Tackle",
    8: "Interception",
    9: "Turnover",
    13: 'Missed',
    14: "Post",
    15: 'Attempt saved',
    16: 'Goal',
    17: 'Card',  # Add a mapping for typeId 17
    44: 'Aerial',  # Add a mapping for typeId 18
    45: 'Dribbled past',
    49: 'Ball Recovery',
    50: 'Dispossessed',
    51: 'Error',
    60: 'Chance missed',
    61: 'Ball Touch',
    
}

# Add the 'contestantId' column to the results DataFrame
results_df['contestantId'] = grouped['contestantId']
#results_df['playerName'] = grouped['playerName']

# Iterate over all typeIDs, renaming them and adding them to the results DataFrame
for typeID in typeIDs_to_count:
    column_name = typeID_to_name.get(typeID, f'Type{typeID}')  # Use the mapped name or a default name
    results_df[column_name] = grouped.get(typeID, 0)  # Use get() to handle missing typeIDs and fill with 0

# Export the results DataFrame to an Excel file
results_df.to_excel('match_data.xlsx', index=False)


In [2]:
import pandas as pd

# Step 2: Read the Excel file into a DataFrame
df = pd.read_excel('/Users/marclambertes/Downloads/ERE2324 - xT.xlsx')

# List of column names you want to count
columns_to_count = [
    "Cross",
    "Cutback",
    "Corner",
    "FK",
    "GK",
    "ThrowIn",
    "Through-ball",
    "LongBall",
    "Switch",
    "Assist",
    "2nd Assist",
    "Number",
    "FromCorner",
    "Keeper throw",
    "GoalMouthY",
    "GoalMouthZ",
    "isOwnGoal",
    "xT",
]

# Identify unique contestant IDs
unique_contestant_ids = df['contestantId'].unique()

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['contestantId'] + columns_to_count)

# Iterate over unique contestant IDs and calculate the counts for each action, including "progressive"
for contestantId in unique_contestant_ids:
    contestant_row = {'contestantId': contestantId}
    for column_name in columns_to_count:
        if column_name == "progressive":
            total_count = df[(df['contestantId'] == contestantId) & (df['progressive'] == True)].shape[0]
        else:
            total_count = df[(df['contestantId'] == contestantId)][column_name].sum()
        contestant_row[column_name] = total_count
    results_df = results_df.append(contestant_row, ignore_index=True)

# Split the "progressive" passes for each contestantId
progressive_counts = df[df['progressive'] == True].groupby('contestantId')['progressive'].count().reset_index()
progressive_counts.rename(columns={'progressive': 'Progressive Pass Count'}, inplace=True)

# Merge the progressive counts with the results DataFrame
results_df = results_df.merge(progressive_counts, on='contestantId', how='left')

# Export the results DataFrame to an Excel file
results_df.to_excel('match_data_pass.xlsx', index=False)


/var/folders/68/mhmqcpdn52943pyql2n4wj440000gn/T/ipykernel_93494/575234527.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(contestant_row, ignore_index=True)
/var/folders/68/mhmqcpdn52943pyql2n4wj440000gn/T/ipykernel_93494/575234527.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(contestant_row, ignore_index=True)
/var/folders/68/mhmqcpdn52943pyql2n4wj440000gn/T/ipykernel_93494/575234527.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(contestant_row, ignore_index=True)
/var/folders/68/mhmqcpdn52943pyql2n4wj440000gn/T/ipykernel_93494/575234527.py:43: FutureWarning: The frame.append method is deprecated and will be remo

In [3]:
import pandas as pd

# Load the Excel files into pandas DataFrames
file1 = pd.read_excel('match_data.xlsx')
file2 = pd.read_excel('match_data_pass.xlsx')

# Merge the DataFrames based on the 'contestantId' column
merged_df = pd.merge(file1, file2, on='contestantId', how='inner')

# Save the merged DataFrame to a new Excel file
merged_df.to_excel('match_data_ERE2324.xlsx', index=False)
